In [2]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb
import data_utils as data
tt = gp.Model('IIITB Exams Timetable')

Restricted license - for non-production use only - expires 2026-11-23


## Make one binary variable for each room-course-timeslot-day combination
$$E_{crtd} = \left\{\begin{array}{ll}1 & \mbox{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$}\\0 & \mbox{Otherwise}\end{array}\right.$$

In [3]:
E = tt.addVars(
    [
        (c,r,t,d)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
        for t in range(data.n_times)
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

## Hard Constraints

In [ ]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course

## Soft Constraints

In [ ]:
# preferred time slots for exams

## Objective Function

In [ ]:
# Maximize Gaps between exams for a student / batch